0. 루브릭
1) Transformer와 비교해 변경이 필요한 부분을 서술하였다.
제출 노트북 파일 앞부분에 텍스트 블럭으로 서술합니다. 변경이 필요한 블럭을 서술합니다.
코드블럭에 변경사항을 주석으로 표시합니다.
2) 모델의 입력 형태에 맞게 전처리를 수행하였다.
Decoder 기반의 생성모델 임을 감안하여 챗봇 데이터를 변형합니다.
3) 모델의 입력 블럭을 GPT 논문에 기반하여 수정하였다.
모델의 input이 정상적으로 구성되었는지 확인합니다.
4) GPT 모델을 정상적으로 구성하였다. (model.summary, model.fit 결과 캡쳐 첨부)
노드의 transformer 코드를 수정하여 GPT1 모델을 구성합니다.
5) 입력에 따른 출력이 생성되었다.
출력 결과물이 수준에 상관없이 모델이 정상적으로 동작하는지 확인합니다.

# Step 1. 데이터 수집하기

패키지 임포트

In [36]:
import pandas as pd
import re
import tensorflow_datasets as tfds
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

### 데이터 로딩: CSV 파일을 로드하고 필요한 데이터로 변환

데이터 로딩: ChatbotData.csv 파일을 로드하고 파싱하는 코드를 추가해야 합니다.
프로젝트의 기존 데이터 로딩 코드는 영화 대화 데이터셋을 로드하는데 사용됩니다. 이를 한국어 챗봇 데이터로 대체해야 합니다.
CSV 파일을 읽어들이고 적절한 형식으로 변환하는 작업이 필요합니다.

# Step 2. 데이터 전처리하기
전처리 함수: 문장에서 필요 없는 문자 제거 및 구두점 처리

In [37]:
df = pd.read_csv('data/ChatbotData.csv')
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [38]:
data = []

for index, row in df.iterrows():
    concatenated_text = f"{row['Q']} $ {row['A']}"
    data.append(concatenated_text)

print(data[:5])

['12시 땡! $ 하루가 또 가네요.', '1지망 학교 떨어졌어 $ 위로해 드립니다.', '3박4일 놀러가고 싶다 $ 여행은 언제나 좋죠.', '3박4일 정도 놀러가고 싶다 $ 여행은 언제나 좋죠.', 'PPL 심하네 $ 눈살이 찌푸려지죠.']


In [39]:
# 전처리 함수
def preprocess_sentence(sentence):
    sentence = sentence.strip()
    # 구두점과 한글 사이에 공백 추가
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    # 한글과 구두점을 제외한 모든 문자를 공백으로 대체
    sentence = re.sub(r"[^가-힣?0-9?.!,]+", " ", sentence) #허용할 부분, 나머지 공백
    sentence = sentence.strip()
    return sentence
print("슝=3")

슝=3


In [40]:
data = list(map(preprocess_sentence, data))
print(data[:5])

['12시 땡 ! 하루가 또 가네요 .', '1지망 학교 떨어졌어 위로해 드립니다 .', '3박4일 놀러가고 싶다 여행은 언제나 좋죠 .', '3박4일 정도 놀러가고 싶다 여행은 언제나 좋죠 .', '심하네 눈살이 찌푸려지죠 .']


모델의 입력으로 사용할 수 있도록 데이터 인코딩

In [41]:
# 시작 토큰과 종료 토큰에 고유한 정수를 부여합니다
# 질문과 답변 데이터셋에 대해서 Vocabulary 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    data, target_vocab_size=2**13)

START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

In [42]:
# 시작 토큰과 종료 토큰을 고려하여 +2를 하여 단어장의 크기를 산정합니다.
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

8351


In [43]:
print('정수 인코딩 후의 21번째 샘플: {}'.format(tokenizer.encode(data[21])))

정수 인코딩 후의 21번째 샘플: [5861, 601, 2483, 4206, 8125, 2350, 7668, 8, 6371, 97, 1]


In [44]:
MAX_LENGTH = 50


In [45]:
def tokenize_and_filter(sentences):
    tokenized = []

    for sentence in sentences:
        sentence = START_TOKEN + tokenizer.encode(sentence) + END_TOKEN

        if len(sentence) <= MAX_LENGTH:
            tokenized.append(sentence)
        
    tokenized = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized, maxlen = MAX_LENGTH, padding = 'post'
    )

    return tokenized

In [46]:
data = tokenize_and_filter(data)
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(data)))

단어장의 크기 : 8351
필터링 후의 질문 샘플 개수: 11823


In [47]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': data[:, :-1]
    },
    {
        'outputs': data[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

트랜스포머 모델 구성

In [48]:
# 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):
    
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model) # position: max_position

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles
    
    def positional_encoding(self, position, d_model):
        # 각도 배열 생성
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)
        
        # 배열의 짝수 인덱스에는 sin 함수 적용
        sines = tf.math.sin(angle_rads[:, 0::2])
        # 배열의 홀수 인덱스에는 cosine 함수 적용
        cosines = tf.math.cos(angle_rads[:, 1::2])

        # sin과 cosine이 교차되도록 재배열
        pos_encoding = tf.stack([sines, cosines], axis=0)
        pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
        pos_encoding = tf.reshape(pos_encoding, [position, d_model])

        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

어텐션

In [49]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
    # 어텐션 가중치는 Q와 K의 닷 프로덕트
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # 가중치를 정규화
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # 패딩에 마스크 추가
    if mask is not None:
        logits += (mask * -1e9)

    # softmax적용
    attention_weights = tf.nn.softmax(logits, axis=-1)

    # 최종 어텐션은 가중치와 V의 닷 프로덕트
    output = tf.matmul(attention_weights, value)
    return output

멀티 헤드 어텐션

In [50]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # Q, K, V에 각각 Dense를 적용합니다
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # 병렬 연산을 위한 머리를 여러 개 만듭니다
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # 스케일드 닷 프로덕트 어텐션 함수
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
        concat_attention = tf.reshape(scaled_attention,
                                          (batch_size, -1, self.d_model))

        # 최종 결과에도 Dense를 한 번 더 적용합니다
        outputs = self.dense(concat_attention)

        return outputs

마스킹

In [51]:
#패딩 마스킹
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [52]:
# 룩 어헤드 마스킹(Look-ahead masking, 다음 단어 가리기) 

# 트랜스포머의 경우에는 문장 행렬을 만들어 한 번에 행렬 형태로 입력으로 들어간다는 특징이 있습니다. 그리고 이 특징 때문에 추가적인 마스킹(Masking) 을 필요
# 전체 문장이 문자 행렬로 들어가서 예측 가능하지만 시도하고자 하는 것은 이전 단어로부터 다음 단어 예측하는 훈련하는것
# 자신보다 다음 나올 단어를 참고하지 않도록 가리는 기법이 룩 어헤드 마스킹 기법
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

디코더 층
encoder-decoder atteion, enc_outputs, padding_mask 제외.


In [53]:
# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    # enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name="look_ahead_mask")
    # padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
    attention1 = MultiHeadAttention(
        d_model, num_heads, name="attention_1")(inputs={
            'query': inputs,
            'key': inputs,
            'value': inputs,
            'mask': look_ahead_mask
        })

    # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    attention1 = tf.keras.layers.Dropout(rate=dropout)(attention1)
    attention1 = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention1 + inputs)

    # # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
    # attention2 = MultiHeadAttention(
    #     d_model, num_heads, name="attention_2")(inputs={
    #         'query': attention1,
    #         'key': enc_outputs,
    #         'value': enc_outputs,
    #         'mask': padding_mask
    #     })

    # # 마스크드 멀티 헤드 어텐션의 결과는
    # # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    # attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    # attention2 = tf.keras.layers.LayerNormalization(
    #     epsilon=1e-6)(attention2 + attention1)

    #마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)제외
    

    # 세 번째 서브 레이어 : 2개의 완전연결층
    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention1)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

    # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(outputs + attention1)

    return tf.keras.Model(
        inputs=[inputs, look_ahead_mask],
        outputs=outputs,
        name=name)

디코더
enc_outputs, padding_mask 제외

In [54]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    # enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name='look_ahead_mask')

    # 패딩 마스크
    # padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
    # 임베딩 레이어
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

    # 포지셔널 인코딩
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    # Dropout이라는 훈련을 돕는 테크닉을 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, look_ahead_mask])

    return tf.keras.Model(
        inputs=[inputs, look_ahead_mask],
        outputs=outputs,
        name=name)

모델 정의 및 학습하기
트랜스포머 함수 정의

인코더 안쓰니까 제외, 디코더 두번째 마스킼 안쓰니까 제외

In [58]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    # dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    # # 인코더에서 패딩을 위한 마스크
    # enc_padding_mask = tf.keras.layers.Lambda(
    #     create_padding_mask, output_shape=(1, 1, None),
    #     name='enc_padding_mask')(inputs)

    # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
    # 내부적으로 패딩 마스크도 포함되어져 있습니다.
    look_ahead_mask = tf.keras.layers.Lambda(
        create_look_ahead_mask,
        output_shape=(1, None, None),
        name='look_ahead_mask')(inputs)

    # # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
    # # 디코더에서 패딩을 위한 마스크
    # dec_padding_mask = tf.keras.layers.Lambda(
    #     create_padding_mask, output_shape=(1, 1, None),
    #     name='dec_padding_mask')(inputs)

    # # 인코더
    # enc_outputs = encoder(
    #     vocab_size=vocab_size,
    #     num_layers=num_layers,
    #     units=units,
    #     d_model=d_model,
    #     num_heads=num_heads,
    #     dropout=dropout,
    # )(inputs=[inputs, enc_padding_mask])

    # 디코더
    dec_outputs = decoder(
        vocab_size=vocab_size,
        num_layers=num_layers,
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
    )(inputs=[inputs, look_ahead_mask])

    # 완전연결층
    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=inputs, outputs=outputs, name=name)

모델 생성

In [59]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
look_ahead_mask (Lambda)        (None, 1, None, None 0           inputs[0][0]                     
__________________________________________________________________________________________________
decoder (Functional)            (None, None, 256)    3192064     inputs[0][0]                     
                                                                 look_ahead_mask[0][0]            
__________________________________________________________________________________________________
outputs (Dense)                 (None, None, 8351)   2146207     decoder[0][0]          

손실 함수

In [60]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

커스템된 학습률

In [61]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)
        
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

모델 컴파일

In [62]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

훈련하기

In [63]:
EPOCHS = 20
early_stopping = EarlyStopping(monitor='loss', patience=3)

history = model.fit(dataset, epochs=EPOCHS, verbose=1, callbacks = [early_stopping])

Epoch 1/20
185/185 [==============================] - 26s 41ms/step - loss: 2.1802 - accuracy: 0.0205
Epoch 2/20
185/185 [==============================] - 8s 41ms/step - loss: 1.9003 - accuracy: 0.0394
Epoch 3/20
185/185 [==============================] - 8s 41ms/step - loss: 1.7177 - accuracy: 0.0436
Epoch 4/20
185/185 [==============================] - 8s 41ms/step - loss: 1.6219 - accuracy: 0.0489
Epoch 5/20
185/185 [==============================] - 8s 42ms/step - loss: 1.5337 - accuracy: 0.0542
Epoch 6/20
185/185 [==============================] - 8s 42ms/step - loss: 1.4478 - accuracy: 0.0585
Epoch 7/20
185/185 [==============================] - 8s 42ms/step - loss: 1.3643 - accuracy: 0.0629
Epoch 8/20
185/185 [==============================] - 8s 41ms/step - loss: 1.2800 - accuracy: 0.0687
Epoch 9/20
185/185 [==============================] - 8s 42ms/step - loss: 1.1910 - accuracy: 0.0754
Epoch 10/20
185/185 [==============================] - 8s 42ms/step - loss: 1.0997 - accur

모델 평가하기

In [67]:
def decoder_inference(sentence):
    sentence = preprocess_sentence(sentence)

    # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
    # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
    sentence = tf.expand_dims(
        START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

    # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
    # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
    output_sequence = tf.expand_dims(START_TOKEN, 0)

    # 디코더의 인퍼런스 단계
    for i in range(MAX_LENGTH):
        # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
        # predictions = model(inputs=[sentence, output_sequence], training=False)
        predictions = model(inputs=sentence, training=False)
        predictions = predictions[:, -1:, :]

        # 현재 예측한 단어의 정수
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
        if tf.equal(predicted_id, END_TOKEN[0]):
            break

        # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
        # 이 output_sequence는 다시 디코더의 입력이 됩니다.
        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

    return tf.squeeze(output_sequence, axis=0)

In [68]:
def sentence_generation(sentence):
    # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
    prediction = decoder_inference(sentence)

    # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
    predicted_sentence = tokenizer.decode(
        [i for i in prediction if i < tokenizer.vocab_size])

    print('입력 : {}'.format(sentence))
    print('출력 : {}'.format(predicted_sentence))

    return predicted_sentence

In [69]:
sentence_generation('오늘 날씨 어때?')

입력 : 오늘 날씨 어때?
출력 : 깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨


'깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨깨'

In [70]:
sentence_generation('깨방정하구나')

입력 : 깨방정하구나
출력 : 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 


'수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 수 '

In [71]:
sentence_generation('그래 끝말잇기하자 수박')

입력 : 그래 끝말잇기하자 수박
출력 : 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 


'로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 로 '

In [72]:
sentence_generation('로미오와 줄리엣')

입력 : 로미오와 줄리엣
출력 :                                                   


'                                                  '

In [73]:
sentence_generation('왜그래? 고장났니?')

입력 : 왜그래? 고장났니?
출력 : 두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두


'두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두두'

In [74]:
sentence_generation('ㅠㅠ')

입력 : ㅠㅠ
출력 : 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 


'하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 하면 '